# CNP_T1_validation_age.ipynb

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Config
toy = False
run_all = False

path = '/home/ubuntu/fmriprep/'
# output_dir = '/output/' #AWS cloud
output_dir = 'data/CNP_T1/' #For Daniel's computer

n_jobs = 20 #amount of cores
cv=4

description = 'CNP_T1_validation_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
# Load data
loaded = np.load(output_dir+'train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']

In [4]:
# loaded = np.load(output_dir+'test_set.npz')
# random_subj_test = loaded['c']
# random_subj_test

# subj_order = np.concatenate([random_subj_train, random_subj_test])
# subj_order

# np.save('final_subj_order', subj_order)


In [5]:
print(y_train.shape)
print(X_train.shape)

(84,)
(84, 1899247)


In [6]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

## Validation 

Grid search returned the best parameters:

* ANOVA k selection: 'all'
* C parameter of SVR: 0.01


In [7]:
from sklearn.pipeline import Pipeline


if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]
    X_validation = X_validation[:8]
    y_validation = y_validation[:8]


# Remove features with too low between-subject variance
# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
anova_svr = Pipeline([
            ('anova', SelectKBest(f_regression, k='all')),
            ('svr', LinearSVR(C=0.01))])

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]


anova_svr.fit(X_train, y_train)
predictions = anova_svr.predict(X_validation)

# clf = SVR(kernel='linear', C=0.01)
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_validation)



score = mean_absolute_error(y_validation, predictions)
r2 = r2_score(y_validation, predictions)
# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:912: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [8]:
print(score)
print(predictions)
print(y_validation)
print(r2)

8.347338196697091
[39.11851421 37.40764335 31.8693247  36.58180737 40.57189757 33.06507724
 29.13717142 17.13445378 36.59497693 45.74497215 34.12581404 29.54060148
 30.4639859  30.36227864 23.09459738 25.20529734 26.39619701 35.52235595
 30.83228595 40.65110628 41.7410892  25.93915221 29.10768664 34.02462882
 27.69351287 35.55288759]
[25 29 40 40 23 26 22 36 29 30 32 23 25 24 35 22 31 48 37 28 41 39 35 23
 32 38]
-0.894329875784931


In [9]:
age_pred = grid.predict(X_train)
mean_abs_error = np.mean(np.abs(age_pred-y_train))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('mean absolute error: '+str(score.round(4))+'\n')
    f.write('r2: '+str(r2.round(4))+'\n')
    f.write(str(predictions)+'\n')
#     f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
#     f.write(str(df1)+'\n')
#     f.write(str(age_pred)+'\n')
#     f.write(str(y_train)+'\n')
#     f.write(str(mean_abs_error)+'\n')
    

NameError: name 'grid' is not defined

In [ ]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [ ]:
# r2 score